In [1]:
import sklearn
assert sklearn.__version__ >= "0.20"
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import feature_column
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras import layers
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df = pd.read_csv('clickbait_data.csv')

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1,random_state=123)

In [4]:
#train.to_csv('clickbait_train.csv',index=False)
#test.to_csv('clickbait_test.csv',index=False)

In [5]:
#df_train = pd.read_csv('clickbait_train.csv')
#df_test = pd.read_csv('clickbait_test.csv')

In [6]:
text = df['headline'].values
labels = df['clickbait'].values

In [7]:
text_train, text_test, y_train, y_test = train_test_split(text, labels,test_size=0.1,random_state=123)

In [8]:
#train_text = df_train['headline'].values
#test_text = df_test['headline'].values

In [9]:
vocab_size = 5000
maxlen = 500
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [23]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
#import xgboost as xgb
#from xgboost import XGBClassifier
from sklearn.cluster import KMeans
#from lightgbm import LGBMClassifier
clf4 = RandomForestClassifier(max_depth=5, random_state=0).fit(X_train, y_train)
y_pred4 = clf4.predict(x_test)

In [22]:
knn = KNeighborsClassifier(n_neighbors=15)
clf = knn.fit(X_train, y_train)
y_pred = clf.predict(x_test)
acc_knb_model=roc_auc_score(y_test, y_pred)*100
acc_knb_model

66.15191543097197

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 500, 32)           8320      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
_________________________________________________________________


In [11]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=1e-4,
                                                  patience=3, verbose=1,
                                                  restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
callbacks = [checkpoint_cb, early_stopping_cb]

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=512, validation_data=(x_test, y_test), epochs=8, callbacks=callbacks)

Epoch 1/8
57/57 [==============================] - 49s 843ms/step - loss: 0.6723 - accuracy: 0.7128 - val_loss: 0.4254 - val_accuracy: 0.9547
Epoch 2/8
57/57 [==============================] - 51s 895ms/step - loss: 0.3155 - accuracy: 0.9514 - val_loss: 0.1572 - val_accuracy: 0.9603
Epoch 3/8
57/57 [==============================] - 50s 870ms/step - loss: 0.1330 - accuracy: 0.9719 - val_loss: 0.1022 - val_accuracy: 0.9725
Epoch 4/8
57/57 [==============================] - 51s 894ms/step - loss: 0.0903 - accuracy: 0.9776 - val_loss: 0.0842 - val_accuracy: 0.9766
Epoch 5/8
57/57 [==============================] - 50s 874ms/step - loss: 0.0622 - accuracy: 0.9857 - val_loss: 0.0773 - val_accuracy: 0.9762
Epoch 6/8
57/57 [==============================] - 51s 892ms/step - loss: 0.0504 - accuracy: 0.9883 - val_loss: 0.0764 - val_accuracy: 0.9775
Epoch 7/8
57/57 [==============================] - 51s 899ms/step - loss: 0.0397 - accuracy: 0.9902 - val_loss: 0.0773 - val_accuracy: 0.9759
Epoch 

In [13]:
tweetTestData = pd.read_csv('tweets-checkpoint.csv')
tweetTestText = [' '.join(map(lambda x: x.strip("\n;[]\\"), l.split(' '))) for l in tweetTestData.postText]

tweetTestLabels = np.round(tweetTestData['isClickbait'].values)

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
predictions = model.predict(x_test)
accuracy_score(y_test,np.round(predictions))

0.97625

In [21]:
max_features = 5000
sequence_length = 500

vectorize_layer = TextVectorization(
#     standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)


In [22]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
])

In [24]:
predictions = export_model.predict(tweetTestText)
accuracy_score(tweetTestLabels,np.round(predictions))

0.7559